In [5]:
# !pip install transformers -U

In [6]:
import os
import gc

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from kaggle_datasets import KaggleDatasets
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

## Helper Functions

In [7]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_mask=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen,
        truncation=True
    )
    
    return np.array(enc_di['input_ids'])

In [8]:
def dict_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_mask=True, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen,
        truncation=True
    )
    
    return {
        "input_ids": np.array(enc_di['input_ids']),
        "attention_mask": np.array(enc_di['attention_mask'])
    }

In [9]:
def build_model(transformer, max_len=512):
    """
    https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
    sequence_output = transformer({"input_ids": input_word_ids, "attention_mask": attention_mask})[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs={
        "input_ids": input_word_ids,
        "attention_mask": attention_mask
    }, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

## TPU Configs

In [10]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


In [11]:
MAX_LEN = 192
MODEL = 'jplu/tf-xlm-roberta-large'
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [22]:
train = pd.read_csv("/kaggle/input/text-classification-int20h/train.csv")

msk = np.random.rand(len(train)) < 0.8
valid = train[~msk]
train = train[msk]


test = pd.read_csv("/kaggle/input/text-classification-int20h/test.csv")
sub = pd.read_csv("/kaggle/input/text-classification-int20h/submission.csv")

n_valid_steps = valid.shape[0] // BATCH_SIZE

In [13]:
n_train_steps = train.shape[0] // BATCH_SIZE

In [14]:
%%time 
x_train = dict_encode(train.review.values, tokenizer, maxlen=MAX_LEN)
y_train = train.sentiment.values

del train
gc.collect()

x_valid = dict_encode(valid.review.values, tokenizer, maxlen=MAX_LEN)
y_valid = valid.sentiment.values
del valid
gc.collect()
x_test = dict_encode(test.review.values, tokenizer, maxlen=MAX_LEN)

CPU times: user 1min 34s, sys: 906 ms, total: 1min 35s
Wall time: 1min 35s


In [15]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            18G        1.4G         10G        1.1M        6.5G         16G
Swap:            0B          0B          0B


In [16]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(4096)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)
del x_train, x_valid, y_train, y_valid
gc.collect()

0

## Load model into the TPU

In [17]:
%%time
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
attention_mask (InputLayer)     [(None, 192)]        0                                            
__________________________________________________________________________________________________
input_ids (InputLayer)          [(None, 192)]        0                                            
__________________________________________________________________________________________________
tf_roberta_model (TFRobertaMode ((None, 192, 1024),  559890432   attention_mask[0][0]             
                                                                 input_ids[0][0]                  
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 1024)]       0           tf_roberta_model[0][0]      

## Train Model

First, we train on the subset of the training set, which is completely in English.

In [18]:
# Configuration
EPOCHS = 2

train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_train_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

Epoch 1/2


/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:431: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  num_elements)


247/247 [==============================] - 172s 696ms/step - loss: 0.3348 - accuracy: 0.8326 - val_loss: 0.1882 - val_accuracy: 0.9269
Epoch 2/2
247/247 [==============================] - 112s 454ms/step - loss: 0.1832 - accuracy: 0.9289 - val_loss: 0.1868 - val_accuracy: 0.9308


Now that we have pretty much saturated the learning potential of the model on english only data, we train it for one more epoch on the `validation` set, which is significantly smaller but contains a mixture of different languages.

In [19]:
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_valid_steps,
    epochs=EPOCHS
)

Epoch 1/2
61/61 [==============================] - 25s 416ms/step - loss: 0.1889 - accuracy: 0.9233
Epoch 2/2
61/61 [==============================] - 25s 415ms/step - loss: 0.1339 - accuracy: 0.9506


In [20]:
model.save_weights("final_weights.h5")

## Submission

In [26]:
sub['sentiment'] = model.predict(test_dataset, verbose=1)
sub.sentiment = (sub.sentiment > 0.5).astype(int)
sub.to_csv('submission.csv', index=False)

79/79 [==============================] - 9s 111ms/step
